In [2]:
# sudoku solver
# 2022-12-30 Mitsuru Yoshizawa
'''
処理概要 

制約単位: 行・列・3x3ブロック
制約:制約単位毎に１から9の数字が入る
候補値表: 9x9の枠毎の候補値集合
処理内容:
(A)繰り返し処理(2,3は制約単位毎に処理)
1. 枠の候補値が1個なら値決定、制約に従い候補値表を更新
2. 候補値に1回しか現れない数字があるなら、その値に決定
3. 複数枠の候補値が同一かつ枠数と候補値数が一致するなら、他の枠の候補値から削除
4. 候補値表に変化がなければ、仮の値を設定して (A)を繰り返す

非常に難易度の高い問題は、仮の値設定が複数回現れます。


使用方法

problem = [
 [0, 9, 0, 0, 0, 0, 0, 3, 0],
 [7, 0, 0, 1, 0, 3, 0, 0, 8],
 [0, 0, 8, 0, 7, 0, 1, 0, 0],
 [0, 0, 9, 0, 0, 0, 5, 0, 0],
 [8, 0, 0, 2, 0, 9, 0, 0, 4],
 [0, 0, 5, 0, 0, 0, 6, 0, 0],
 [0, 0, 1, 0, 8, 0, 2, 0, 0],
 [6, 0, 0, 5, 0, 2, 0, 0, 1],
 [0, 3, 0, 0, 0, 0, 0, 4, 0]]

sudoku_solver(problem)

'''
import numpy as np
import copy

def sudoku_solver(problem, debug=False):
    prob = np.array(problem).astype(np.int32)

    if prob.shape != (9,9):
        print('問題が 9 x 9 になっていません')
        return
    
    if prob.min()<0 or 9<prob.max():
        print('値は0から9の整数にしてください')
        return
    
    board = prob.copy()
    print('問題')
    print(board)
    
    # board_numsテーブルの初期化 セル毎に可能性のある数字の集合を示す
    board_nums = np.full((9,9),set())
    for r in range(0,9):
        for c in range(0,9):
            board_nums[r][c]=set(range(1,10))
    
    for r in range(0,9):
        for c in range(0,9):
            if board[r][c]>0:
                decide_val(board_nums, board, r, c, board[r][c],debug=False)
    
    if debug:
        print(board_nums)
    
    # create group
    group = []
    for r in range(0,9):
        group.append([(r,c) for c in range(0,9)])
    for c in range(0,9):
        group.append([(r,c) for r in range(0,9)])
    for mr in range(0,9,3):
        for mc in range(0,9,3):
            group.append([(mr+i,mc+j) for i in range(0, 3) for j in range(0, 3)])          
    
    ret, ret_board = iterate_process(group, board_nums, board,debug=debug)
        
    print('処理終了 回答')
    print(ret_board)

        
def iterate_process(group, board_nums, board,debug=False):
    for i in range(1, 100):
        if debug:
            print("Loop :",i)

        before_board_nums = board_nums.copy()
        
        if detect_unique_num(group, board_nums, board,debug=debug):
            if debug:
                print('(A) detect_unique_num')
            continue
        
        if decide_multi_cells_vals(group, board_nums, board,debug=debug):
            if debug:
                print('(B) decide_multi_cells_vals')
            continue
            
        if detect_single_num(group, board_nums, board,debug=debug):
            if debug:
                print('(C) detect_single_num')
            continue

        if np.count_nonzero(board_nums == set() )==81:
            if ck_result(group, board):
                print('** 成功 **')
                # print('board')
                # print(board)
                return True, copy.deepcopy(board)
            else:
                # print('** Error 問題がおかしい可能性もある **')
                return False, copy.deepcopy(board)

        if np.array_equal(before_board_nums, board_nums):
            # if debug:
            print('-- 仮置実施 --')  
            # 最も少ない候補値を持つセルに対して、候補値を1つづつ試してゆく
            n_min,rc, nums = 9, None, None
            for r in range(0,9):
                for c in range(0,9):
                    if len(board_nums[r][c])>1 and len(board_nums[r][c])<n_min:
                        n_min = len(board_nums[r][c])
                        rc = [r,c]
                        nums = tuple(board_nums[r][c])
            if debug:
                print(f'仮 {tuple(rc)} vals:{nums}')
            
            for val in nums:
                if debug:
                    print(f'仮 val={val}')
                    print(board)
                    print(board_nums)
                bk_board = copy.deepcopy(board)
                bk_board_nums = copy.deepcopy(board_nums)
                
                decide_val(bk_board_nums, bk_board, rc[0], rc[1], val, debug=debug)
                
                # recursion
                ret, ret_board= iterate_process(group, bk_board_nums, bk_board,debug=debug)
                if ret:
                    if debug:
                        print('-- 成功しました --')
                    return True, copy.deepcopy(ret_board)

            if debug:
                print('-- 失敗しました --')
            return False, copy.deepcopy(board)


def decide_multi_cells_vals(group, board_nums, board,debug=False):
# 3✖️3マトリックス,行, 列ごとに、複数セルのセル候補値が同一かつそのセル数と候補値数が一致するなら、他のセルの候補値ではない

    for g in group:
        wk = {}
        for rc in g:
            r,c = rc[0], rc[1]
            if len(board_nums[r][c])>1:
                wk[tuple(board_nums[r][c])] = []
                
        for rc in g:
            r,c = rc[0], rc[1]
            if len(board_nums[r][c])>1:
                wk[tuple(board_nums[r][c])].append((r,c))

        for k in wk.keys():
            if len(k)==len(wk[k]) and len(k)>1:
                for rc in g:
                    r,c = rc[0], rc[1]
                    if set(k) != board_nums[r][c]:
                        before = board_nums[r][c].copy()
                        board_nums[r][c]-= set(k)
                        if before != board_nums[r][c]:
                            if debug:
                                print(f'decide_multi_cells_vals ({r},{c}) {before} -->{board_nums[r][c]}')
                            return True
    return False

# 行, 列, 3✖️3マトリックスごとの候補値に、1回しか現れない数字があれば、その値に決定
def detect_single_num(group, board_nums, board, debug=False):    
    for g in group:
        cnts = {x:[] for x in range(1,10)}           
        for rc in g:
            for n in board_nums[rc[0]][rc[1]]:
                cnts[n].append(rc)

        for n in range(1,10):
            if len(cnts[n])==1:
                rs = cnts[n][0][0]
                cs = cnts[n][0][1]
                decide_val(board_nums, board, rs, cs, n, debug=debug, reason='Single Num')
                return True
    return False

def detect_unique_num(group, board_nums, board, debug=False):
    for g in group:
        for rc in g:
            if len(board_nums[rc[0]][rc[1]])==1:  # 候補が1つしかない
                val = list(board_nums[rc[0]][rc[1]])[0]
                decide_val(board_nums, board, rc[0], rc[1], val, debug=debug, reason='Unique Num')
                return True
    return False

def decide_val(board_nums, board, r, c, val, debug=False, reason=None):
# board_numsの更新処理
# 行, 列, 3✖️3マトリックスごとに、すでにセル値が定まっている場合に、他のセルの候補値集合からその値を除く
    
    def _remove_val(r,c, val):
        before = board_nums[r][c].copy()
        board_nums[r][c].discard(val)   # 他のセルの候補値集合からその値を除く
        
        if debug and before != board_nums[r][c]:
            print(f'_remove_val ({r},{c}) {before} -->{board_nums[r][c]}')
     
    if board[r][c]>0 and board[r][c]!=val:
        print(f'Error set_val_and_upd_boardnums ({r},{c}) before val:{board[r][c]} after val:{val}')

    board[r][c] = val
    board_nums[r][c] = set()
    
    if debug:
        print(f'Set ({r},{c}) val={val} reason={reason}')
    
    # 行
    for cs in range(0,9):
        if cs != c:
            _remove_val(r,cs, val)
        
    # 列
    for rs in range(0,9):
        if rs != r:
            _remove_val(rs,c, val)
    
    # 3 x 3 matrix
    rm = r - r%3
    cm = c - c%3
    for i in range(0,3):
        for j in range(0,3):
            rs = rm + i
            cs = cm + j
            if rs != r and cs != c:
                _remove_val(rs,cs, val)

def ck_result(group, board):
    # sudokuのルールに沿ったチェック
    for g in group:
        lis = [ board[rc[0], rc[1]] for rc in g if board[rc[0], rc[1]]>0 ]
        if not(len(lis)==9 and len(set(lis))==9):
            return False
    return True


In [7]:
# 読売 2022.11.26
problem = [
 [1, 0, 0, 8, 0, 0, 6, 0, 0],
 [0, 8, 0, 0, 0, 0, 4, 2, 0],
 [0, 0, 7, 0, 4, 2, 0, 0, 0],
 [0, 5, 0, 0, 0, 8, 0, 0, 6],
 [8, 0, 0, 0, 0, 0, 0, 0, 2],
 [4, 0, 0, 6, 0, 0, 0, 9, 0],
 [0, 0, 0, 2, 9, 0, 5, 0, 0],
 [0, 9, 6, 0, 0, 0, 0, 1, 0],
 [0, 0, 4, 0, 0, 5, 0, 0, 3]]

# sudoku_solver(problem, debug=True)
sudoku_solver(problem)

問題
[[1 0 0 8 0 0 6 0 0]
 [0 8 0 0 0 0 4 2 0]
 [0 0 7 0 4 2 0 0 0]
 [0 5 0 0 0 8 0 0 6]
 [8 0 0 0 0 0 0 0 2]
 [4 0 0 6 0 0 0 9 0]
 [0 0 0 2 9 0 5 0 0]
 [0 9 6 0 0 0 0 1 0]
 [0 0 4 0 0 5 0 0 3]]
-- 仮置実施 --
** 成功 **
処理終了 回答
[[1 4 2 8 7 9 6 3 5]
 [9 8 5 3 6 1 4 2 7]
 [6 3 7 5 4 2 1 8 9]
 [7 5 1 9 2 8 3 4 6]
 [8 6 9 4 1 3 7 5 2]
 [4 2 3 6 5 7 8 9 1]
 [3 1 8 2 9 6 5 7 4]
 [5 9 6 7 3 4 2 1 8]
 [2 7 4 1 8 5 9 6 3]]


In [4]:
# Minimum Sudoku 2919
problem = [
 [0, 0, 0, 0, 1, 0, 6, 0, 0],
 [3, 0, 0, 0, 0, 0, 0, 2, 0],
 [7, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 7, 0, 2, 0, 0, 0],
 [0, 1, 0, 0, 0, 0, 8, 0, 0],
 [5, 0, 0, 3, 0, 0, 0, 0, 0],
 [0, 0, 0, 2, 0, 0, 0, 3, 5],
 [4, 0, 0, 0, 0, 0, 0, 0, 7],
 [0, 6, 0, 0, 0, 0, 0, 0, 0]]

# sudoku_solver(problem, debug=True)
sudoku_solver(problem)

問題
[[0 0 0 0 1 0 6 0 0]
 [3 0 0 0 0 0 0 2 0]
 [7 0 0 0 0 0 0 0 0]
 [0 0 0 7 0 2 0 0 0]
 [0 1 0 0 0 0 8 0 0]
 [5 0 0 3 0 0 0 0 0]
 [0 0 0 2 0 0 0 3 5]
 [4 0 0 0 0 0 0 0 7]
 [0 6 0 0 0 0 0 0 0]]
-- 仮置実施 --
-- 仮置実施 --
-- 仮置実施 --
** 成功 **
処理終了 回答
[[8 2 4 5 1 7 6 9 3]
 [3 5 1 6 4 9 7 2 8]
 [7 9 6 8 2 3 5 1 4]
 [9 4 8 7 6 2 3 5 1]
 [6 1 3 9 5 4 8 7 2]
 [5 7 2 3 8 1 9 4 6]
 [1 8 9 2 7 6 4 3 5]
 [4 3 5 1 9 8 2 6 7]
 [2 6 7 4 3 5 1 8 9]]


In [6]:
# The hardest sudokus  Rating 99529
problem = [
 [1, 2, 0, 4, 0, 0, 3, 0, 0],
 [3, 0, 0, 0, 1, 0, 0, 5, 0],
 [0, 0, 6, 0, 0, 0, 1, 0, 0],
 [7, 0, 0, 0, 9, 0, 0, 0, 0],
 [0, 4, 0, 6, 0, 3, 0, 0, 0],
 [0, 0, 3, 0, 0, 2, 0, 0, 0],
 [5, 0, 0, 0, 8, 0, 7, 0, 0],
 [0, 0, 7, 0, 0, 0, 0, 0, 5],
 [0, 0, 0, 0, 0, 0, 0, 9, 8]]

# sudoku_solver(problem, debug=True)
sudoku_solver(problem)

問題
[[1 2 0 4 0 0 3 0 0]
 [3 0 0 0 1 0 0 5 0]
 [0 0 6 0 0 0 1 0 0]
 [7 0 0 0 9 0 0 0 0]
 [0 4 0 6 0 3 0 0 0]
 [0 0 3 0 0 2 0 0 0]
 [5 0 0 0 8 0 7 0 0]
 [0 0 7 0 0 0 0 0 5]
 [0 0 0 0 0 0 0 9 8]]
-- 仮置実施 --
-- 仮置実施 --
-- 仮置実施 --
-- 仮置実施 --
-- 仮置実施 --
-- 仮置実施 --
-- 仮置実施 --
-- 仮置実施 --
-- 仮置実施 --
-- 仮置実施 --
-- 仮置実施 --
-- 仮置実施 --
-- 仮置実施 --
-- 仮置実施 --
-- 仮置実施 --
-- 仮置実施 --
-- 仮置実施 --
-- 仮置実施 --
-- 仮置実施 --
-- 仮置実施 --
-- 仮置実施 --
-- 仮置実施 --
-- 仮置実施 --
-- 仮置実施 --
-- 仮置実施 --
-- 仮置実施 --
-- 仮置実施 --
-- 仮置実施 --
-- 仮置実施 --
-- 仮置実施 --
-- 仮置実施 --
-- 仮置実施 --
-- 仮置実施 --
-- 仮置実施 --
-- 仮置実施 --
-- 仮置実施 --
-- 仮置実施 --
-- 仮置実施 --
-- 仮置実施 --
-- 仮置実施 --
-- 仮置実施 --
-- 仮置実施 --
-- 仮置実施 --
-- 仮置実施 --
-- 仮置実施 --
-- 仮置実施 --
-- 仮置実施 --
-- 仮置実施 --
-- 仮置実施 --
-- 仮置実施 --
-- 仮置実施 --
-- 仮置実施 --
-- 仮置実施 --
-- 仮置実施 --
-- 仮置実施 --
-- 仮置実施 --
-- 仮置実施 --
-- 仮置実施 --
-- 仮置実施 --
-- 仮置実施 --
-- 仮置実施 --
-- 仮置実施 --
-- 仮置実施 --
-- 仮置実施 --
-- 仮置実施 --
-- 仮置実施 --
-- 仮置実施 --
-- 仮置実施 --
-- 仮置実施 --
-- 仮置実施 --
-- 仮置実施 --
-- 仮置実施 --
-- 仮置実施 --
-- 仮